# Rheostat 3 Continued 
## Prep for murray mtg

In this notebook, we will try to take a look at comparing the dynamics with/without the rheostat and take a look at the rate of atp depletion.

In [1]:
#Using Bioscrape: Basic Imports

#A Model is a CRN with some bells and whistles
from bioscrape.types import Model

#py_simulate_model is a helper function that takes care of may details for you
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
import pylab as plt

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
#import bokeh_catplot


bokeh.io.output_notebook()

Loading BokehJS ...

Let's make a function to help us with the plotting.

In [2]:
def plot(species, parameters, reactions, t_max, x_0, title, legend_location = 'center_right'):
    
    M = Model(species = species, reactions = reactions, parameters = parameters,
         initial_condition_dict = x_0)
    timepoints = np.linspace(0,t_max,1000)
    df_results = py_simulate_model(timepoints, Model = M, stochastic = False)
    
    # First plot glucose and isobutanol
    p = bokeh.plotting.figure(width = 300, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration',
                             title = title)
    p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
    p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
    p.line(timepoints, df_results['f16p'], color = 'purple', legend_label = 'f16p')
    p.legend.location = legend_location

    # Plot ATP and ADP, NADPH, NADP+
    p2 = bokeh.plotting.figure(width = 300, height = 250,
                               x_axis_label = 'time',
                             y_axis_label = 'concentration')
    p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
    p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
    p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
    p2.legend.location = legend_location
    

    # Show plots
    bokeh.io.show(row(p, p2))
    
    # CALCULATE AND SHOW MAX ATP AND TIME
    # get atp values
    atp_vals = list(df_results['atp'].values)
    # get rid of initial value
    atp_vals.remove(max(atp_vals))
    # return max value now
    max_atp = max(atp_vals)
    print('The maximum ATP value is:', max_atp)
    time = atp_vals.index(max_atp)
    print('The time at which it occurs is:', timepoints[time])

# Simplified Model

In [3]:
species = ['atp', 'glucose', 'adp', 'pi', 'f16p', 'isobutanol']

parameters = [('kf', 1), ('kr', 0.1), ('delta', 10),
             ('k_glu', 0.01)]

rxn1 = (['atp', 'atp', 'glucose'], ['adp', 'adp', 'f16p'], 'massaction', {'k': 'kf'})
rxn2 = (['adp', 'adp', 'f16p'], ['atp', 'atp', 'isobutanol'], 'massaction', {'k': 'kf'})
rxn3 = (['adp', 'adp', 'adp', 'pi', 'f16p'], ['atp', 'atp', 'atp', 'isobutanol'], 'massaction', {'k': 'kf'})
rxn_supp = ([], ['glucose'], 'massaction', {'k':'k_glu'})

rxn_d_atp = (['atp'], ['adp', 'pi'], 'massaction', {'k': 'delta'})


reactions = [rxn1, rxn2, rxn3, rxn_d_atp, rxn_supp]

x_0 = {
    'glucose': 100,
    'atp':200
}

t_max = 1
title = 'Simplified Rheostat Model'

In [4]:
plot(species, parameters, reactions, t_max, x_0, title)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The following species are uninitialized and their value has been defaulted to 0: adp, pi, f16p, isobutanol, 
  after removing the cwd from sys.path.


The maximum ATP value is: 182.24459227828896
The time at which it occurs is: 0.037037037037037035


Look at just the stoichiometric model:

In [5]:
reactions = [rxn1, rxn2, rxn_d_atp]
title = 'Simplified Stoichiometric Model'

plot(species, parameters, reactions, t_max, x_0, title)
print('The maximum ATP value is :', 1.30599215e+02 )
print('The time at which it occurs is :', 0.03903903903903904)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The following species are uninitialized and their value has been defaulted to 0: adp, pi, f16p, isobutanol, 
  after removing the cwd from sys.path.


The maximum ATP value is: 165.37020939624247
The time at which it occurs is: 0.0
The maximum ATP value is : 130.599215
The time at which it occurs is : 0.03903903903903904


# 1 Step Model

In [13]:
species = ['atp', 'glucose', 'e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p',
           'e2', '2adp:f16p:e2','2atp:isobutanol:e2', 
           '3adppi:f16p:e3', 'e3' , '3atp:isobutanol:e3', 'isobutanol']


parameters = [('kf', 10), ('kr', 0.1), ('delta', 10)]


# reaction 1
rxn1_f = (['atp', 'atp', 'glucose', 'e1'], ['2atp:glucose:e1'],
           'massaction',  {'k': 'kf'})
rxn1_r = ( ['2atp:glucose:e1'],['atp', 'atp', 'glucose', 'e1'], 
            'massaction', {'k': 'kr'})
# reaction 2
rxn2_f = ( ['2atp:glucose:e1'],['2adp:f16p:e1'], 
           'massaction', {'k': 'kf'})
# reaction 3
rxn3_f =  ( ['2adp:f16p:e1'], ['adp', 'adp', 'f16p', 'e1'], 
           'massaction', {'k': 'kf'})
rxn3_r =  (['adp', 'adp', 'f16p', 'e1'], ['2adp:f16p:e1'],
           'massaction', {'k': 'kr'})
# reaction 4
rxn4_f =  (['f16p', 'adp', 'adp', 'e2'], ['2adp:f16p:e2'],
           'massaction', {'k': 'kf'})
rxn4_r =  ( ['2adp:f16p:e2'], ['f16p', 'adp', 'adp', 'e2'],
           'massaction', {'k': 'kr'})
# reaction 5 
rxn5_f =  (['2adp:f16p:e2'], ['2atp:isobutanol:e2'],
           'massaction', {'k': 'kf'})
# reaction 6
rxn6_f =  (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol', 'e2'],
           'massaction', {'k': 'kf'})
rxn6_r =  (['atp', 'atp', 'isobutanol', 'e2'], ['2atp:isobutanol:e2'],
           'massaction', {'k': 'kr'})
# reaction 7
rxn7_f = ( ['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'],['3adppi:f16p:e3'], 
              'massaction', {'k': 'kf'})
rxn7_r = ( ['3adppi:f16p:e3'],['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'], 
              'massaction', {'k': 'kr'})
# reaction 8
rxn8_f = ( ['3adppi:f16p:e3'],['3atp:isobutanol:e3'], 
              'massaction', {'k': 'kf'})
# reaction 9
rxn9_f = ( ['3atp:isobutanol:e3'],['atp', 'atp','atp',  'isobutanol', 'e3'], 
              'massaction', {'k': 'kf'})
rxn9_r = ( ['atp', 'atp', 'atp', 'isobutanol', 'e3'],['3atp:isobutanol:e3'], 
              'massaction', {'k': 'kr'})
# ATP DEGRADATION
rxn_d_atp = (["atp"], ['adp', 'pi'], "massaction", {"k":"delta"})

reactions = [rxn1_f, rxn1_r, rxn2_f, rxn3_f, rxn3_r,rxn4_f,
             rxn4_r, rxn5_f, rxn6_f,  rxn6_r, rxn7_f,
             rxn7_r, rxn8_f, rxn9_f, rxn9_r, rxn_d_atp]
# Initial Conditions             
x_0 = {
    "glucose":100,
    "atp":200, 
    'e1': 200, 
    'e2':200,
    'e3':200
}

t_max = 10

title = '1 Step Rheostat Model'

plot(species, parameters, reactions, t_max, x_0, title, legend_location = 'top_left')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The following species are uninitialized and their value has been defaulted to 0: 2atp:glucose:e1, 2adp:f16p:e1, adp, pi, f16p, 2adp:f16p:e2, 2atp:isobutanol:e2, 3adppi:f16p:e3, 3atp:isobutanol:e3, isobutanol, 
  after removing the cwd from sys.path.


The maximum ATP value is: 1.574167115158717
The time at which it occurs is: 0.18018018018018017


Now, let's look at stoichiometric version of the 1 step model.

In [12]:
reactions = [rxn1_f, rxn1_r, rxn2_f, rxn3_f, rxn3_r,rxn4_f,
             rxn4_r, rxn5_f, rxn6_f,  rxn6_r, rxn7_f,
            rxn_d_atp]

title = '1 Step Stoichiometric Model'
plot(species, parameters, reactions, t_max, x_0, title, legend_location = 'top_left')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The following species are uninitialized and their value has been defaulted to 0: 2atp:glucose:e1, 2adp:f16p:e1, adp, pi, f16p, 2adp:f16p:e2, 2atp:isobutanol:e2, 3adppi:f16p:e3, 3atp:isobutanol:e3, isobutanol, 
  after removing the cwd from sys.path.
odeint failed with mxstep=500...

The maximum ATP value is: 2.453651980050531
The time at which it occurs is: 0.0


# Three Step Model Version 1

In [8]:
species = ['glucose', 'e1', 'atp', 'glucose:e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p:e1', 'f16p',
          'e2', 'f16p:e2', '2adp:f16p:e2', '2atp:isobutanol:e2', 'isobutanol:e2',
          'e3', 'f16p:e3', '3adppi:f16p:e3', '3atp:isobutanol:e3', 'isobutanol:e3', 'isobutanol']

parameters = [('k_f', 10), ('k_r', 0.1), ('delta', 10)]

rxn1_f = (['glucose', 'e1'], ['glucose:e1'], 'massaction', {'k': 'k_f'})
rxn1_r = (['glucose:e1'], ['glucose', 'e1'], 'massaction', {'k': 'k_r'})

rxn2_f = (['atp', 'atp', 'glucose:e1'], ['2atp:glucose:e1'], 'massaction', {'k': 'k_f'})
rxn2_r = (['2atp:glucose:e1'], ['atp', 'atp', 'glucose:e1'], 'massaction', {'k': 'k_r'})

rxn3_f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_f'})

rxn4_f = (['2adp:f16p:e1'], ['adp', 'adp', 'f16p:e1'], 'massaction', {'k': 'k_f'})
rxn4_r = (['adp', 'adp', 'f16p:e1'], ['2adp2pi:f16p:e1'], 'massaction', {'k': 'k_r'})

rxn5_f = (['f16p:e1'], ['f16p', 'e1'], 'massaction', {'k': 'k_f'})

rxn6_f = (['f16p', 'e2'], ['f16p:e2'], 'massaction', {'k': 'k_f'})
rxn6_r = (['f16p:e2'], ['f16p', 'e2'], 'massaction', {'k': 'k_r'})

rxn7_f = (['adp', 'adp','f16p:e2'], ['2adp:f16p:e2'], 'massaction', {'k': 'k_f'})
rxn7_r = (['2adp2pi:f16p:e2'], ['adp', 'adp', 'pi', 'pi', 'f16p:e2'], 'massaction', {'k': 'k_r'})

rxn8_f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_f'})

rxn9_f = (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol:e2'], 'massaction', {'k': 'k_f'})
rxn9_r = (['atp', 'atp', 'isobutanol:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_r'})

rxn10_f = (['isobutanol:e2'], ['isobutanol', 'e2'], 'massaction', {'k': 'k_f'})
rxn10_r = (['isobutanol', 'e2'], ['isobutanol:e2'], 'massaction', {'k': 'k_r'})

rxn11_f = (['f16p', 'e3'], ['f16p:e3'], 'massaction', {'k': 'k_f'})
rxn11_r = ( ['f16p:e3'], ['f16p', 'e3'], 'massaction', {'k': 'k_r'})

rxn12_f = (['adp', 'adp', 'adp', 'pi', 'f16p:e3'], ['3adppi:f16p:e3'], 'massaction', {'k': 'k_f'})
rxn12_r = (['3adppi:f16p:e3'], ['adp', 'adp', 'adp', 'pi', 'f16p:e3'], 'massaction', {'k': 'k_r'})

rxn13_f = (['3adppi:f16p:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_f'})

rxn14_f = (['3atp:isobutanol:e3'], ['atp', 'atp', 'atp', 'isobutanol:e3'], 'massaction', {'k': 'k_f'})
rxn14_r = ( ['atp', 'atp', 'atp', 'isobutanol:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_r'})

rxn15_f = (['isobutanol:e3'], ['isobutanol', 'e3'], 'massaction', {'k': 'k_f'})
rxn15_r = (['isobutanol', 'e3'], ['isobutanol:e3'], 'massaction', {'k': 'k_r'})

rxn_d_atp = (["atp"], ['adp', 'pi'], "massaction", {"k":"delta"})


reactions = [rxn1_f, rxn1_r, rxn2_f, rxn2_r, rxn3_f, rxn4_f, rxn4_r, rxn5_f, rxn6_f, rxn6_r, rxn7_f, rxn7_r,
            rxn8_f, rxn9_f, rxn9_r, rxn10_f, rxn10_r, rxn11_f, rxn11_r, rxn12_f, rxn12_r, rxn13_f,
            rxn14_f, rxn14_r, rxn15_f, rxn15_r, rxn_d_atp]

x_0 = {
    'glucose': 100,
    'atp':200,
    'e1':200,
    'e2':200,
    'e3':200
}

t_max = 10

In [9]:
title = '3 Step V1 Rheostat Model'
plot(species, parameters, reactions, t_max, x_0, title)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The following species are uninitialized and their value has been defaulted to 0: glucose:e1, 2atp:glucose:e1, 2adp:f16p:e1, adp, pi, f16p:e1, f16p, f16p:e2, 2adp:f16p:e2, 2atp:isobutanol:e2, isobutanol:e2, f16p:e3, 3adppi:f16p:e3, 3atp:isobutanol:e3, isobutanol:e3, isobutanol, 2adp2pi:f16p:e1, 2adp2pi:f16p:e2, 
  after removing the cwd from sys.path.


The maximum ATP value is: 27.289747299083796
The time at which it occurs is: 0.5005005005005005


In [10]:
reactions = [rxn1_f, rxn1_r, rxn2_f, rxn2_r, rxn3_f, rxn4_f, rxn4_r, rxn5_f, rxn6_f, rxn6_r, rxn7_f, rxn7_r,
            rxn8_f, rxn9_f, rxn9_r, rxn10_f, rxn10_r, rxn_d_atp]

title = '3 Step V1 Stoichiometric Model'
plot(species, parameters, reactions, t_max, x_0, title)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The following species are uninitialized and their value has been defaulted to 0: glucose:e1, 2atp:glucose:e1, 2adp:f16p:e1, adp, pi, f16p:e1, f16p, f16p:e2, 2adp:f16p:e2, 2atp:isobutanol:e2, isobutanol:e2, f16p:e3, 3adppi:f16p:e3, 3atp:isobutanol:e3, isobutanol:e3, isobutanol, 2adp2pi:f16p:e1, 2adp2pi:f16p:e2, 
  after removing the cwd from sys.path.


The maximum ATP value is: 22.041531388605954
The time at which it occurs is: 0.46046046046046046


# Three Step Model Version 2

In [63]:
species = ['e1', 'atp', 'e1:2atp', '2atp:glucose:e1', '2adp:f16p:e1', '2adp:e1', 'f16p', '2adp:e1', 'adp', 'pi',
          'e2', 'e2:2adp', '2adp:f16p:e2', '2atp:isobutanol:e2', '2atp:e2', 'isobutanol',
          'e3', 'e2:3adppi', '3adppi:f16p:e3', '3atp:f16p:e3', '3atp:e3']

parameters = [('kf', 10), ('kr', 0.1), ('delta', 10)]

rxn1f = (['e1', 'atp', 'atp'], ['e1:2atp'], 'massaction', {'k': 'kf'})
rxn1r = (['e1:2atp'], ['e1', 'atp', 'atp'], 'massaction', {'k': 'kf'})

rxn2f = (['glucose', 'e1:2atp'], ['2atp:glucose:e1'], 'massaction', {'k': 'kf'})
rxn2r = (['2atp:glucose:e1'], ['glucose', 'e1:2atp'], 'massaction', {'k': 'kr'})

rxn3f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'kf'})

rxn4f = (['2adp:f16p:e1'], ['2adp:e1', 'f16p'], 'massaction', {'k': 'kf'})
rxn4r = (['2adp:e1', 'f16p'], ['2adp:f16p:e1'], 'massaction', {'k': 'kr'})

rxn5f = (['2adp:e1'], ['adp', 'adp', 'e1'], 'massaction', {'k': 'kf'})
rxn5r = (['adp', 'adp', 'e1'], ['2adp:e1'], 'massaction', {'k': 'kr'})

rxn6f = (['e2', 'adp', 'adp'], ['e2:2adp'], 'massaction', {'k': 'kf'})
rxn6r = (['e2:2adp'], ['e2', 'adp', 'adp'], 'massaction', {'k': 'kf'})

rxn7f = (['f16p', 'e2:2adp'], ['2adp:f16p:e2'], 'massaction', {'k': 'kf'})
rxn7r = (['2adp:f16p:e2'], ['f16p', 'e2:2adp'], 'massaction', {'k': 'kf'})

rxn8f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'kf'})

rxn9f = (['2atp:isobutanol:e2'], ['2atp:e2', 'isobutanol'], 'massaction', {'k': 'kf'})
rxn9r = (['2atp:e2', 'isobutanol'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'kr'})

rxn10f = (['2atp:e2'], ['atp', 'atp', 'e2'], 'massaction', {'k': 'kf'})
rxn10r = (['atp', 'atp', 'e2'], ['2atp:e2'], 'massaction', {'k': 'kr'})

rxn11f = (['e3', 'adp', 'adp', 'adp', 'pi'], ['e3:3adppi'], 'massaction', {'k': 'kf'})
rxn11r = (['e3:3adppi'], ['e3', 'adp', 'adp', 'adp', 'pi'], 'massaction', {'k': 'kr'})

rxn12f = (['f16p', 'e3:3adppi'], ['3adppi:f16p:e3'], 'massaction', {'k': 'kf'})
rxn12r = (['3adppi:f16p:e3'], ['f16p', 'e3:3adppi'], 'massaction', {'k': 'kr'})

rxn13f = (['3adppi:f16p:e3'], ['3atp:f16p:e3'], 'massaction', {'k': 'kf'})

rxn14f = (['3atp:f16p:e3'], ['3atp:e3', 'f16p'], 'massaction', {'k': 'kf'})
rxn14r = (['3atp:e3', 'f16p'], ['3atp:f16p:e3'], 'massaction', {'k': 'kr'})

rxn15f = (['3atp:e3'], ['atp', 'atp', 'atp', 'e3'], 'massaction', {'k': 'kf'})
rxn15r = (['atp', 'atp', 'atp', 'e3'], ['3atp:e3'], 'massaction', {'k': 'kr'})

rxn_d_atp = (["atp"], ['adp', 'pi'], "massaction", {"k":"delta"})

reactions = [rxn1f, rxn1r, rxn2f, rxn2r, rxn3f, rxn4f, rxn4r, rxn5f, rxn5r, rxn6f, rxn6r, rxn7f, rxn7r,
            rxn8f, rxn9f, rxn9r, rxn10f, rxn10r, rxn11f, rxn11r, rxn12f, rxn12r, rxn13f, rxn14f, rxn14r,
            rxn15f, rxn15r, rxn_d_atp]

x_0 = {
    'glucose': 100,
    'atp':200,
    'e1':200,
    'e2':200,
    'e3':200
}

In [64]:
t_max = 10
title = '3 Step V2 Rheostat Model'
plot(species, parameters, reactions, t_max, x_0, title)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The following species are uninitialized and their value has been defaulted to 0: e1:2atp, 2atp:glucose:e1, 2adp:f16p:e1, 2adp:e1, f16p, adp, pi, e2:2adp, 2adp:f16p:e2, 2atp:isobutanol:e2, 2atp:e2, isobutanol, e2:3adppi, 3adppi:f16p:e3, 3atp:f16p:e3, 3atp:e3, e3:3adppi, 
  after removing the cwd from sys.path.


The maximum ATP value is: 0.8552284856450925
The time at which it occurs is: 3.6436436436436437


In [65]:
reactions = [rxn1f, rxn1r, rxn2f, rxn2r, rxn3f, rxn4f, rxn4r, rxn5f, rxn5r, rxn6f, rxn6r, rxn7f, rxn7r,
            rxn8f, rxn9f, rxn9r, rxn10f, rxn10r, rxn_d_atp]
title = '3 Step V2 Stoichiometric Model'
plot(species, parameters, reactions, t_max, x_0, title)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The following species are uninitialized and their value has been defaulted to 0: e1:2atp, 2atp:glucose:e1, 2adp:f16p:e1, 2adp:e1, f16p, adp, pi, e2:2adp, 2adp:f16p:e2, 2atp:isobutanol:e2, 2atp:e2, isobutanol, e2:3adppi, 3adppi:f16p:e3, 3atp:f16p:e3, 3atp:e3, 
  after removing the cwd from sys.path.


The maximum ATP value is: 0.8290507142546079
The time at which it occurs is: 1.6416416416416417


# Analysis
Let's try to code some functions that will help us quantify differences.

Let's look at the max ATP and at what time it occurs.